# BG/NBD model

Comparison with lifetimes

**Reference**: Fader, P. S., Hardie, B. G., & Lee, K. L. (2005). “Counting your customers” the easy way: An alternative to the Pareto/NBD model. Marketing science, 24(2), 275-284.

http://www.brucehardie.com/papers/bgnbd_2004-04-20.pdf

In [1]:
import arviz as az
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from pymc_marketing import clv

In [3]:
from lifetimes.datasets import load_cdnow_summary
from lifetimes import BetaGeoFitter

In [4]:
df = load_cdnow_summary(index_col=[0])
df.head()

,frequency,recency,T
ID,,,
1,2,30.43,38.86
2,1,1.71,38.86
3,0,0.00,38.86
4,0,0.00,38.86
5,0,0.00,38.86


In [15]:
data = pd.DataFrame({
    'customer_id': np.arange(len(df['frequency'].values)),
    'frequency': df["frequency"].values,
    'recency': df["recency"].values,
    'T': df['T'].values,
})


In [16]:
model = clv.BetaGeoModel(data = data)
model.build_model()
model.fit()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [a, b, alpha, r]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 10 seconds.
/Users/michalraczycki/Documents/pymc-marketing/pymc_marketing/clv/models/basic.py:116: UserWarning: The effect of Potentials on other parameters is ignored during posterior predictive sampling. This is likely to lead to invalid or biased predictive samples.
  idata.extend(pm.sample_posterior_predictive(idata))
/Users/michalraczycki/Documents/pymc-marketing/.conda/envs/marketing_env/lib/python3.11/site-packages/arviz/data/inference_data.py:1459: UserWarning: The group fit_data is not defined in the InferenceData scheme
  warnings.warn(


Inference data with groups:
	> posterior
	> sample_stats
	> fit_data

In [17]:
model.fit_summary()

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
a,0.993,0.301,0.544,1.516,0.009,0.007,1337.0,1227.0,1.0
b,3.274,1.277,1.508,5.535,0.040,0.031,1350.0,1283.0,1.0
alpha,4.479,0.391,3.778,5.223,0.010,0.007,1651.0,2133.0,1.0
r,0.244,0.013,0.220,0.268,0.000,0.000,1620.0,1785.0,1.0


In [18]:
model.fit_summary()

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
a,0.993,0.301,0.544,1.516,0.009,0.007,1337.0,1227.0,1.0
b,3.274,1.277,1.508,5.535,0.040,0.031,1350.0,1283.0,1.0
alpha,4.479,0.391,3.778,5.223,0.010,0.007,1651.0,2133.0,1.0
r,0.244,0.013,0.220,0.268,0.000,0.000,1620.0,1785.0,1.0


### Using MAP fit

`CLVModel`s, which includes `BetaGeoModel`, can provide the maximum a posteriori estimates using a numerical optimzer from `scipy.optimize`.

In [19]:
model.fit(fit_method="map") # results from previous fit are overriden

/Users/michalraczycki/Documents/pymc-marketing/.conda/envs/marketing_env/lib/python3.11/site-packages/arviz/data/inference_data.py:1459: UserWarning: The group fit_data is not defined in the InferenceData scheme
  warnings.warn(


Inference data with groups:
	> posterior
	> fit_data

In [20]:
model.fit_summary()

a        0.793
b        2.426
alpha    4.414
r        0.243
Name: value, dtype: float64

### Comparing with the `lifetimes` package

In [21]:
bgf = BetaGeoFitter()
bgf.fit(frequency=data['frequency'].values, recency=data['recency'].values, T=data['T'].values)
bgf.summary

,coef,se(coef),lower 95% bound,upper 95% bound
r,0.242593,0.012557,0.217981,0.267205
alpha,4.413532,0.378221,3.672218,5.154846
a,0.792886,0.185719,0.428877,1.156895
b,2.425752,0.705345,1.043276,3.808229
